In [1]:
import pandas as pd
import os
import re
import json
import string
from collections import Counter
from itertools import chain
from importlib import reload

from tqdm import tqdm
tqdm.pandas(desc="progress: ")

# Chinese character set
from zhon import hanzi
import opencc

# Utility variable
import sys
sys.path.insert(0, '../..')

# var
import var.var as V
import var.path as P

# utils
import utils.data as D
import utils.io as IO
import utils.get_path as GP
import utils.preprocess as PP

In [2]:
cc = opencc.OpenCC('s2tw')

In [3]:
df_applications = D.read_df_applications()
df_recommendation_letters = D.read_df_recommendation_letters()
df_achievements = D.read_df_achievements()

In [ ]:
df_applications.head()

In [ ]:
df_recommendation_letters.head()

In [ ]:
df_achievements.head()

In [7]:
from docx import Document

In [8]:
def get_ach_ss_rl(_year, _id):
    ## process achievements
    row = df_achievements.query('`year` == @_year and `id` == @_id')
    ach = row['achievement'].to_list()
    ach = '。'.join(ach) + '。'
    ## process self statement and recommendation letter
    row = df_applications.query('`id` == @_id')
    
    ss = row['self_statement'].to_list()[0][1:]
    ss = ''.join(ss)
    
    rl_rows = df_recommendation_letters.query('`year` == {} and `id` == {}'.format(_year, _id))
    try:
        rls_sents = rl_rows['all_paragraph_sent'].to_list()
    except:
        rls_sents = []

    if rls_sents == None:
        rls_sents = []
    
    rls_sents = [''.join(sents) for sents in rls_sents]
    rl = '。'.join(rls_sents)
    
    text = '\n'.join([ach, ss, rl])
    text = text.replace('\n', '')
    text = "generate summary for the following article：" + text
    
    print(text)
    
    return text

In [9]:
def get_ach_ss(_year, _id):
    ## process achievements
    row = df_achievements.query('`year` == @_year and `id` == @_id')
    ach = row['achievement'].to_list()
    ach = '。'.join(ach) + '。'
    ## process self statement and recommendation letter
    row = df_applications.query('`id` == @_id')
    
    ss = row['self_statement'].to_list()[0][1:]
    ss = ''.join(ss)
    
    text = '\n'.join([ach, ss])
    text = text.replace('\n', '')
    text = "generate summary for the following article：" + text
    
    print(text)
    
    return text

In [10]:
def get_rl(_year, _id):
    ## process self statement and recommendation letter
    rl_rows = df_recommendation_letters.query('`year` == {} and `id` == {}'.format(_year, _id))
    try:
        rls_sents = rl_rows['all_paragraph_sent'].to_list()
    except:
        rls_sents = []

    if rls_sents == None:
        rls_sents = []
    
    rls_sents = [''.join(sents) for sents in rls_sents]
    rl = '。'.join(rls_sents)
    
    text = '\n'.join([rl])
    text = text.replace('\n', '')
    text = "generate summary for the following article：" + text
    
    print(text)
    
    return text

In [55]:
df_chatgpt_test = pd.read_csv("112_F_experiment.csv")

In [ ]:
df_chatgpt_test

In [ ]:
for _, row in df_chatgpt_test.iterrows():
    _year = row['year']
    _id = row['id']
    _name = row['name']
    _group = row['group']
    
    _app = get_ach_ss(_year, _id)
    
    doc = Document()
    
    fn = "{}_{}_{}.docx".format(_year, _id, _name)
    _group_dir = os.path.join("../../experiment/112_F_quantitative_experiment", _group, "app")
    
    if not os.path.exists(_group_dir):
        os.makedirs(_group_dir)
    
    doc.add_paragraph(_app)
    doc.save(os.path.join(_group_dir, fn))

## Convert ChatGPT summary to traditional chinese

In [ ]:
# for _, row in df_chatgpt_test.iterrows():
#     _year = row['year']
#     _id = row['id']
#     _name = row['name']
#     _group = row['group']
#     _summary = row['summary']
    
#     _summary = cc.convert(_summary)
    
#     doc = Document()
    
#     fn = "{}_{}_{}.docx".format(_year, _id, _name)
#     _group_dir = os.path.join("../../112_F_ChatGPT", _group, "ChatGPT_summary")
    
#     if not os.path.exists(_group_dir):
#         os.makedirs(_group_dir)
    
#     doc.add_paragraph(_summary)
#     doc.save(os.path.join(_group_dir, fn))
    
#     doc = Document()
    
#     fn = "{}_{}_{}_app.docx".format(_year, _id, _name)
#     _group_dir = os.path.join(_group, "app")
    
#     if not os.path.exists(_group_dir):
#         os.makedirs(_group_dir)
    
#     doc.save(os.path.join(_group_dir, fn))

## Save the results

In [39]:
D.write_df_applications(df_applications, file='csv')
D.write_df_applications(df_applications, file='pkl')